# The Chat Format
In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

In [1]:
!pip install --quiet panel jupyter_bokeh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.7 MB/s eta 0:00:00


In [2]:
import os
import google.generativeai as genai

from google.colab import userdata
import google.generativeai as genai

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
def get_completion(prompt, model="gemini-1.5-pro"):
    generation_config = {
        "temperature": 0,
        "top_p": 1,
        "top_k": 1,
        "max_output_tokens": 2048,
    }

    model = genai.GenerativeModel(
        model_name=model,
        generation_config=generation_config,
    )

    response = model.generate_content(prompt)
    return response.text

def get_completion_from_messages(messages, model_name="gemini-1.5-pro", temperature=0):
    generation_config = {
        "temperature": temperature,
        "top_p": 0.95,
        "top_k": 64,
        "max_output_tokens": 8192,
    }

    model = genai.GenerativeModel(
        model_name=model_name,
        generation_config=generation_config,
    )

    chat = model.start_chat()
    current_response = None

    for msg in messages:
        if msg['role'] == 'user':
            current_response = chat.send_message(msg['content'])
        elif msg['role'] == 'system':
            next_user_msg = next((m['content'] for m in messages[messages.index(msg):] if m['role'] == 'user'), None)
            if next_user_msg:
                combined_msg = f"{msg['content']}\n\nUser: {next_user_msg}"
                current_response = chat.send_message(combined_msg)

    return current_response.text if current_response else "No user messages found."

In [4]:
messages =  [
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},
{'role':'user', 'content':'tell me a joke'},
{'role':'assistant', 'content':'Why did the chicken cross the road'},
{'role':'user', 'content':'I don\'t know'}  ]

In [5]:
response = get_completion_from_messages(messages, temperature=1)
print("Response:", response)

Response: Fear not, good gentle soul, if knowledge doth escape thee!  Even the wisest of owls hath a moment when the mouse scurries from their grasp. What troubles thee? Perchance I can offer aid, though my wisdom be but a flickering candle in the vast darkness. Speak freely, and let thy burden be shared.



# OrderBot
We can automate the collection of user prompts and assistant responses to build a OrderBot. The OrderBot will take orders at a pizza restaurant

In [10]:
import panel as pn
import google.generativeai as genai

# Initialize Panel
pn.extension()

# Conversation context with a system message defining the chatbot's behavior.
context = [{
    'role': 'system',
    'content': (
        "You are OrderBot, an automated service to collect orders for a pizza restaurant. "
        "You first greet the customer, then collect the order, and then ask if it's a pickup or delivery. "
        "You wait to collect the entire order, then summarize it and check for a final time if the customer wants "
        "to add anything else. If it's a delivery, you ask for an address. Finally, you collect the payment. "
        "Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu. "
        "You respond in a short, very conversational friendly style. The menu includes: "
        "pepperoni pizza 12.95, 10.00, 7.00; cheese pizza 10.95, 9.25, 6.50; "
        "eggplant pizza 11.95, 9.75, 6.75; fries 4.50, 3.50; greek salad 7.25; "
        "Toppings: extra cheese 2.00, mushrooms 1.50, sausage 3.00, canadian bacon 3.50, AI sauce 1.50, peppers 1.00; "
        "Drinks: coke 3.00, 2.00, 1.00; sprite 3.00, 2.00, 1.00; bottled water 5.00."
    )
}]

def get_completion_from_messages(messages, model_name="gemini-2.0-flash", temperature=0):
    """
    Sends the conversation messages to the Gemini  model and returns the AI's response.
    """
    model = genai.GenerativeModel(model_name=model_name)
    chat = model.start_chat()
    current_response = None

    for msg in messages:
        if msg['role'] == 'user':
            current_response = chat.send_message(msg['content'])
        elif msg['role'] == 'system':
            next_user_msg = next(
                (m['content'] for m in messages[messages.index(msg):] if m['role'] == 'user'),
                None
            )
            if next_user_msg:
                combined_msg = f"{msg['content']}\n\nUser: {next_user_msg}"
                current_response = chat.send_message(combined_msg)

    return current_response.text if current_response else "No user messages found."

def collect_messages(event):
    """
    Retrieves the user input, updates the conversation context, gets the AI response, and updates the UI.
    """
    user_input = inp.value.strip()
    if not user_input:
        return

    # Clear the input widget
    inp.value = ""

    # Update the conversation context with the user's message
    context.append({'role': 'user', 'content': user_input})

    # Get the AI's response
    response = get_completion_from_messages(context)

    # Update the conversation context with the assistant's message
    context.append({'role': 'assistant', 'content': response})

    # Update the conversation display
    conversation_panel.append(
        pn.Row("User:", pn.pane.Markdown(user_input, width=600))
    )
    conversation_panel.append(
        pn.Row("Assistant:", pn.pane.Markdown(response, width=600))
    )

# Create UI widgets
inp = pn.widgets.TextInput(value="Hi", placeholder="Enter text here...")
button_conversation = pn.widgets.Button(name="Chat!")
button_conversation.on_click(collect_messages)
conversation_panel = pn.Column()

# Create the dashboard layout
dashboard = pn.Column(
    inp,
    button_conversation,
    conversation_panel
)

# Display the dashboard (in a Jupyter notebook) or serve it as an app.
dashboard

Column
    [0] TextInput(placeholder='Enter text here...', value='Hi')
    [1] Button(name='Chat!')
    [2] Column()

### How can we get this into a POS?

In [ ]:
def get_completion_from_messages_summary(messages, temperature=0):
    model = genai.GenerativeModel(model_name="gemini-1.5-pro")
    # Start a new chat without history
    chat = model.start_chat(history=[])

    # Send all messages at once as context
    conversation_text = "Conversation history:\n"
    for msg in messages[:-1]:  # Exclude the last system prompt
        conversation_text += f"{msg['role']}: {msg['content']}\n"

    # Add the request for JSON summary
    json_prompt = messages[-1]['content']
    final_prompt = f"{conversation_text}\n{json_prompt}"

    response = chat.send_message(final_prompt)
    return response.text

# Now use this function for the summary
messages = context.copy()
messages.append(
    {'role':'system', 'content':"""
    Based on the conversation above, create a JSON summary of the order with:
    {
      "pizzas": [],
      "toppings": [],
      "drinks": [],
      "sides": [],
      "total_price": "",
      "delivery_info": {
        "address": "",
        "payment_method": ""
      }
    }
    Only respond with the JSON.
    """}
)

response = get_completion_from_messages_summary(messages, temperature=0)
print(response)

```json
{
  "pizzas": [],
  "toppings": [],
  "drinks": [],
  "sides": [],
  "total_price": "",
  "delivery_info": {
    "address": "",
    "payment_method": ""
  }
}
```



## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!